In [16]:
import numpy as np
import pandas as pd
import random

In [17]:
df = pd.read_csv('data/data.csv', sep=";", index_col=0)
df_coords = pd.read_csv('data/coords.csv', sep=";", index_col=0)

In [18]:
df_coords.iloc[0][1]

24.96540995

In [19]:
#Distansmatrix. Y-led är startpunkt, X-led stoppunkt

df

,Arcada,Arabia,Itis,Sello,Kaari,Myyrmanni,Lauttis,Citycenter,Columbus,Lippulaiva,Redi,Dixi
Arcada,0.0,0.6,10.4,12.8,16.2,12.7,9.5,5.6,14.5,22.2,2.9,16.3
Arabia,0.6,0.0,10.2,12.7,8.3,12.5,9.3,5.4,14.4,22.0,2.7,16.1
Itis,10.8,10.7,0.0,18.5,13.4,17.9,16.4,11.5,4.2,33.8,8.9,13.6
Sello,13.6,13.4,18.8,0.0,6.4,9.0,11.0,14.5,22.9,17.0,13.8,19.2
Kaari,11.4,10.4,13.6,5.7,0.0,6.2,15.0,9.8,17.7,21.0,11.6,14.1
Myyrmanni,12.6,14.5,18.0,8.7,7.4,0.0,17.9,13.9,22.2,23.9,14.5,17.2
Lauttis,8.6,8.4,13.9,12.5,12.2,16.4,0.0,3.7,18.1,16.0,7.2,22.1
Citycenter,5.3,5.2,10.2,10.9,9.8,14.0,4.8,0.0,14.4,17.5,3.5,19.3
Columbus,14.8,14.6,4.1,22.5,17.4,21.9,20.3,15.4,0.0,37.7,12.8,15.4
Lippulaiva,22.1,22.5,33.8,16.8,21.4,23.9,13.7,17.3,37.9,0.0,20.7,34.2


In [20]:
mallNames = {
    0:"Arcada",
    1:"Arabia",
    2:"Itis",
    3:"Sello",
    4:"Kaari",
    5:"Myyrmanni",
    6:"Lauttis",
    7:"Citycenter",
    8:"Columbus",
    9:"Lippulaiva",
    10:"Redi",
    11:"Dixi"
}

In [21]:
def generatePopulation(size):
    population = []
    
    for i in range(size):
        chromosome = random.sample(range(1,12), 11)
        chromosome.append(0) # Sätt in nolla (fa tillbaka till Arcada)
        chromosome.insert(0, 0) # Sätt in nolla till början (Börja alltid från Arcada)
        #print(chromosome)
        
        population.append(chromosome)
        
    return population
    
    

In [22]:
generatePopulation(5)

[[0, 8, 11, 2, 10, 4, 5, 9, 1, 7, 3, 6, 0],
 [0, 10, 8, 4, 5, 7, 11, 3, 9, 2, 1, 6, 0],
 [0, 9, 11, 4, 1, 10, 2, 6, 5, 3, 8, 7, 0],
 [0, 3, 9, 2, 6, 10, 5, 1, 7, 4, 11, 8, 0],
 [0, 2, 5, 3, 8, 10, 4, 1, 11, 6, 9, 7, 0]]

In [23]:
df.iloc[10][7]
df['Citycenter']['Redi']

4.3

In [24]:
def displayFitness(chromosome):
    distance = 0
    route_text = ""
    # Displaya distans (km) och rutten
    for i, gene in enumerate(chromosome):
        # Ta distansen från df distans matrix och addera ihop
        try:
            distance += df.iloc[chromosome[i+1]][gene]
            route_text += mallNames[gene] + " -> "
            
        except:
            #Rutten tar slut när vi kommer fram till error
            route_text += "Arcada"
            pass
        
    print(distance)
    print(route_text)

In [25]:
def fitness(population):
    fitnessScore = []
    
    for chromosome in population:
        score = 0
        distance = 0
        #print(type(chromosome))
        
        #Kalkylera distans och ge score       
        for i, gene in enumerate(chromosome):
            # Ta distansen från df distans matrix och addera ihop
            try:
                distance += df.iloc[chromosome[i+1]][gene]
            except:
                #Rutten tar slut när vi kommer fram till error
                pass
            #print(distance)
            
        # Om rutten är illegal, dvs. start och slut inte vid Arcada, ge penalty.
        if not (chromosome[0] == 0) or not (chromosome[12] == 0):
            distance = distance * 100
            
        #fitnessScore[distance] = chromosome
        fitnessScore.append(1 / distance)
        #print("Distance:", distance)
        
    return fitnessScore
            

In [26]:
# credits: https://youtu.be/nhT56blfRpE
def selection(population, fitness_score, retain):
    # använder random.choices för att hitta två "bästa" alternativen från  population och fitness score
    # K = antalet output arrays (=parents)
     return random.choices(
        population = population,
        weights = fitness_score,
        k=retain
    )
    

In [27]:
#####
# Credits: https://stackoverflow.com/a/72511644
# för PMX crossover
#####
def pmx_crossover(parent1, parent2):
    
    # Konvertera från list till numpy array
    parent1 = np.array(parent1)
    parent2 = np.array(parent2)
    
    # Initsialisera barnet
    offspring = np.zeros(len(parent1))

    # Välj slumpmässig sekvens mellan parent1[1] och parent1[11]
    # dvs. vi väljer aldrig start och stopp punkterna för crossover, utan mellanstoppen
    rng = np.random.default_rng()
    random_start, random_end = np.sort(rng.choice(np.arange(1 , len(parent1)), size=2, replace=False))
    # DEBUG PRINT
    #print(offspring)
    # Kopiera sekvens från parent1 till offspring
    offspring[random_start:random_end] = parent1[random_start:random_end]
    # DEBUG PRINT
    #print(offspring)
    
    for i in np.concatenate([np.arange(random_start), np.arange(random_end,len(parent1))]):
        candidate = parent2[i]
        while candidate in parent1[random_start:random_end]: # allows for several successive mappings
            #print(f"Candidate {candidate} not valid in position {i}") # DEBUGONLY
            candidate = parent2[np.where(parent1 == candidate)[0][0]]
        offspring[i] = candidate
        #print(offspring)
    
    
    return offspring.astype(int)
    
    
    
pmx_crossover([0, 10, 7, 4, 6, 3, 2, 1, 5, 9, 11, 8, 0], [0, 3, 5, 10, 6, 1, 11, 9, 2, 8, 7, 4, 0])

array([ 0, 10,  7,  4,  6,  3,  2,  9, 11,  8,  5,  1,  0])

In [28]:
# Inversion
def mutation(child, mutation_rate):
    #print("Original:", child)
    if mutation_rate > random.random():
        start = random.randint(1, len(child)-1)
        stop = random.randint(start, len(child)-1)
        
        genes = child[start:stop]
        genes = np.flip(genes)
        
        i = 0
        if len(genes) > 0:
            for gene in range(start, stop):
                child[gene] = genes[i]
                i += 1
        
    return child

#mutation([0, 10, 7, 4, 6, 3, 2, 1, 5, 9, 11, 8, 0], 1)
    

In [29]:
def PMX_crossover(parent1, parent2, seed):
    '''
    parent1 and parent2 are 1D np.array
    '''
    rng = np.random.default_rng(seed=seed)

    cutoff_1, cutoff_2 = np.sort(rng.choice(np.arange(len(parent1)+1), size=2, replace=False))

    def PMX_one_offspring(p1, p2):
        offspring = np.zeros(len(p1), dtype=p1.dtype)

        # Copy the mapping section (middle) from parent1
        offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

        # copy the rest from parent2 (provided it's not already there
        for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
            candidate = p2[i]
            while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                print(f"Candidate {candidate} not valid in position {i}") # DEBUGONLY
                candidate = p2[np.where(p1 == candidate)[0][0]]
            offspring[i] = candidate
        return offspring

    offspring1 = PMX_one_offspring(parent1, parent2)
    offspring2 = PMX_one_offspring(parent2, parent1)

    return offspring1, offspring2

#Credits: https://stackoverflow.com/a/72511644

In [30]:
numOfPopulation = 150
mutationRate = 0
rounds = 500
bestIndividual = []
bestScore = 0

retain = int(numOfPopulation * 30/100)
retain = numOfPopulation - retain

population = generatePopulation(numOfPopulation)

for round in range(rounds):
    
    fit_s = fitness(population)
    
    index_max = np.argmax(fit_s)
    if fit_s[index_max] > bestScore:
        bestScore = fit_s[index_max]
        bestIndividual = population[index_max]
        print("-----")
        print("New best individual with score", bestScore)
        print(bestIndividual)
        displayFitness(bestIndividual)
        print("-----")
    
    parents = selection(population, fit_s, retain)
    
    numOfParents = len(parents)
    numOfChildren = numOfPopulation - numOfParents
    
    children = []
    
    
    while len(children) < numOfChildren:
        
        randParent1 = random.randint(0, numOfParents-1)
        randParent2 = random.randint(0, numOfParents-1)
        
        children.append(pmx_crossover(parents[randParent1], parents[randParent2]))
        
    for i in range(numOfChildren):
        children[i] = mutation(children[i], mutationRate)
        
    population = parents + children
    fit_s = np.sort(fit_s)
    #print(fit_s)


-----
New best individual with score 0.007204610951008645
[0, 1, 7, 9, 4, 5, 2, 8, 10, 6, 3, 11, 0]
138.8
Arcada -> Arabia -> Citycenter -> Lippulaiva -> Kaari -> Myyrmanni -> Itis -> Columbus -> Redi -> Lauttis -> Sello -> Dixi -> Arcada
-----
-----
New best individual with score 0.007692307692307693
[ 0  1 10  6  7  3  5  9  4 11  2  8  0]
130.0
Arcada -> Arabia -> Redi -> Lauttis -> Citycenter -> Sello -> Myyrmanni -> Lippulaiva -> Kaari -> Dixi -> Itis -> Columbus -> Arcada
-----
-----
New best individual with score 0.008006405124099279
[ 0  1  3  5  4  7  6  9 10  8  2 11  0]
124.90000000000002
Arcada -> Arabia -> Sello -> Myyrmanni -> Kaari -> Citycenter -> Lauttis -> Lippulaiva -> Redi -> Columbus -> Itis -> Dixi -> Arcada
-----
-----
New best individual with score 0.008285004142502071
[ 0  1 10  7  4  3  6  9  5 11  8  2  0]
120.7
Arcada -> Arabia -> Redi -> Citycenter -> Kaari -> Sello -> Lauttis -> Lippulaiva -> Myyrmanni -> Dixi -> Columbus -> Itis -> Arcada
-----
-----
New 